In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_parquet("./data/clean_data_merged_v1.parquet")
data['HomeTeamName'] = data['HomeTeamName'].str.strip()
data['AwayTeamName'] = data['AwayTeamName'].str.strip()

#data = data[data['date'].dt.year >= 1980].copy()
data['year'] = data['date'].dt.year

In [3]:
data.head(5)

,date,HomeTeamName,AwayTeamName,HomeTeamGoals,AwayTeamGoals,y_classif,y_reg,year
0,1960-07-06 20:00:00,France,Yugoslavia,4,5,-1.0,-1,1960
1,1960-07-06 21:30:00,Czechoslovakia,Soviet Union,0,3,-1.0,-3,1960
2,1960-07-09 21:30:00,Czechoslovakia,France,2,0,1.0,2,1960
3,1960-07-10 21:30:00,Soviet Union,Yugoslavia,2,1,1.0,1,1960
4,1964-06-17 20:00:00,Spain,Hungary,2,1,1.0,1,1964


In [4]:
participationh = data.groupby(["HomeTeamName", "year"]).size()#.reset_index()
participationa = data.groupby(["AwayTeamName", "year"]).size()#.reset_index()

participationh.index.names = ['team', 'year']
participationa.index.names = ['team', 'year']

participation = pd.DataFrame({"home": participationh, "away": participationa}).fillna(0).sum(axis=1).reset_index()
participation.columns = ['team', 'year', 'n_games']

participation = participation.sort_values("year")
participation['cumulative_games'] = participation.groupby("team")['n_games'].cumsum()

participation['year'] = participation['year'] + 1

# stats um time contra o outro

In [5]:
def fn_features(df, team_features):
    df_ = pd.merge_asof(df, team_features, left_by=['HomeTeamName'], right_by=['team'], on=['year'])
    df_ = pd.merge_asof(df_, team_features, left_by=['AwayTeamName'], right_by=['team'], on=['year'], suffixes=['_home', '_away'])
    
    return df_

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, log_loss, roc_auc_score

In [9]:
val_

,date,HomeTeamName,AwayTeamName,HomeTeamGoals,AwayTeamGoals,y_classif,y_reg,year,team_home,n_games_home,cumulative_games_home,team_away,n_games_away,cumulative_games_away
0,2016-06-10 21:00:00,France,Romania,2,1,1.0,1,2016,France,4.0,32.0,Romania,3.0,13.0
1,2016-06-11 15:00:00,Albania,Switzerland,0,1,-1.0,-1,2016,NaN,NaN,NaN,Switzerland,3.0,9.0
2,2016-06-15 18:00:00,Romania,Switzerland,1,1,0.0,0,2016,Romania,3.0,13.0,Switzerland,3.0,9.0
3,2016-06-15 21:00:00,France,Albania,2,0,1.0,2,2016,France,4.0,32.0,NaN,NaN,NaN
4,2016-06-19 21:00:00,Romania,Albania,0,1,-1.0,-1,2016,Romania,3.0,13.0,NaN,NaN,NaN
5,2016-06-19 21:00:00,Switzerland,France,0,0,0.0,0,2016,Switzerland,3.0,9.0,France,4.0,32.0
6,2016-06-11 18:00:00,Wales,Slovakia,2,1,1.0,1,2016,NaN,NaN,NaN,NaN,NaN,NaN
7,2016-06-11 21:00:00,England,Russia,1,1,0.0,0,2016,England,4.0,27.0,Russia,3.0,14.0
8,2016-06-15 15:00:00,Russia,Slovakia,1,2,-1.0,-1,2016,Russia,3.0,14.0,NaN,NaN,NaN
9,2016-06-16 15:00:00,England,Wales,2,1,1.0,1,2016,England,4.0,27.0,NaN,NaN,NaN


In [7]:
for year in [2004, 2008, 2012, 2016]:
    tr = data[data['year'] < year].copy()
    val = data[data['year'] == year].copy()
    
   
    selected_features = ['n_games_home', 'n_games_away', 'cumulative_games_home', 'cumulative_games_away']
    
    tr_ = fn_features(tr, participation)
    val_ = fn_features(val, participation)
    
    
    Xtr = tr_[selected_features].fillna(0)
    Xval = val_[selected_features].fillna(0)
    ytr = tr['y_classif']
    yval = val['y_classif']
    
    mdl = LogisticRegression()
    mdl.fit(Xtr, ytr)
    
    p = mdl.predict_proba(Xval)
    
    
        
    baseline = ytr.value_counts(normalize=True)
    baseline.index = [2,0,1]
    baseline = np.ones((Xval.shape[0], 3)) * baseline.sort_index().values
    
    
    print("Ano: {} | Baseline Log Loss: {:.4f} | Baseline AUC: {:.4f} | Log Loss: {:.4f} | ROC AUC: {:.4f}\n".format(year, log_loss(yval, baseline),
            roc_auc_score(yval, baseline, multi_class='ovr'), log_loss(yval, p), roc_auc_score(yval,p, multi_class='ovr')))
    


Ano: 2004 | Baseline Log Loss: 1.1082 | Baseline AUC: 0.5000 | Log Loss: 1.2580 | ROC AUC: 0.4980

Ano: 2008 | Baseline Log Loss: 1.0736 | Baseline AUC: 0.5000 | Log Loss: 1.0453 | ROC AUC: 0.6075

Ano: 2012 | Baseline Log Loss: 1.0708 | Baseline AUC: 0.5000 | Log Loss: 1.0026 | ROC AUC: 0.6375

Ano: 2016 | Baseline Log Loss: 1.0898 | Baseline AUC: 0.5000 | Log Loss: 1.0900 | ROC AUC: 0.5874



2004 Baseline Home Win 0.3548387096774194
              precision    recall  f1-score   support

        -1.0       0.19      0.30      0.23        10
         0.0       0.00      0.00      0.00        10
         1.0       0.33      0.45      0.38        11

    accuracy                           0.26        31
   macro avg       0.17      0.25      0.21        31
weighted avg       0.18      0.26      0.21        31

2004               precision    recall  f1-score   support

        -1.0       0.27      0.40      0.32        10
         0.0       0.00      0.00      0.00        10
         1.0       0.44      0.64      0.52        11

    accuracy                           0.35        31
   macro avg       0.23      0.35      0.28        31
weighted avg       0.24      0.35      0.29        31



2008 Baseline Historical wins 0.41935483870967744
2008 Baseline Home Win 0.3548387096774194
              precision    recall  f1-score   support

        -1.0       0.47      0.53      0.50        15
         0.0       0.00      0.00      0.00         5
         1.0       0.36      0.45      0.40        11

    accuracy                           0.42        31
   macro avg       0.28      0.33      0.30        31
weighted avg       0.35      0.42      0.38        31


2008               precision    recall  f1-score   support

        -1.0       0.62      0.53      0.57        15
         0.0       0.17      0.20      0.18         5
         1.0       0.42      0.45      0.43        11

    accuracy                           0.45        31
   macro avg       0.40      0.40      0.40        31
weighted avg       0.47      0.45      0.46        31


2012 Baseline Historical wins 0.2903225806451613
2012 Baseline Home Win 0.41935483870967744
              precision    recall  f1-score   support

        -1.0       0.30      0.55      0.39        11
         0.0       0.00      0.00      0.00         7
         1.0       0.27      0.23      0.25        13

    accuracy                           0.29        31
   macro avg       0.19      0.26      0.21        31
weighted avg       0.22      0.29      0.24        31

2012               precision    recall  f1-score   support

        -1.0       0.41      0.82      0.55        11
         0.0       0.00      0.00      0.00         7
         1.0       0.67      0.46      0.55        13

    accuracy                           0.48        31
   macro avg       0.36      0.43      0.36        31
weighted avg       0.42      0.48      0.42        31


2016 Baseline Historical wins 0.4117647058823529
2016 Baseline Home Win 0.39215686274509803
              precision    recall  f1-score   support

        -1.0       0.36      0.76      0.49        17
         0.0       0.00      0.00      0.00        14
         1.0       0.53      0.40      0.46        20

    accuracy                           0.41        51
   macro avg       0.30      0.39      0.32        51
weighted avg       0.33      0.41      0.34        51


2016               precision    recall  f1-score   support

        -1.0       0.42      0.47      0.44        17
         0.0       0.00      0.00      0.00        14
         1.0       0.47      0.75      0.58        20

    accuracy                           0.45        51
   macro avg       0.30      0.41      0.34        51
weighted avg       0.32      0.45      0.37        51

In [104]:
val_

,date,HomeTeamName,AwayTeamName,HomeTeamGoals,AwayTeamGoals,y_classif,y_reg,year,team_home,n_games_home,cumulative_games_home,team_away,n_games_away,cumulative_games_away
0,2004-06-12 17:00:00,Portugal,Greece,1,2,-1.0,-1,2004,Portugal,5.0,13.0,Greece,3.0,3.0
1,2004-06-12 19:45:00,Spain,Russia,1,0,1.0,1,2004,Spain,4.0,21.0,Russia,3.0,3.0
2,2004-06-16 17:00:00,Greece,Spain,1,1,0.0,0,2004,Greece,3.0,3.0,Spain,4.0,21.0
3,2004-06-16 19:45:00,Russia,Portugal,0,2,-1.0,-2,2004,Russia,3.0,3.0,Portugal,5.0,13.0
4,2004-06-20 19:45:00,Spain,Portugal,0,1,-1.0,-1,2004,Spain,4.0,21.0,Portugal,5.0,13.0
5,2004-06-20 19:45:00,Russia,Greece,2,1,1.0,1,2004,Russia,3.0,3.0,Greece,3.0,3.0
6,2004-06-13 17:00:00,Switzerland,Croatia,0,0,0.0,0,2004,Switzerland,3.0,3.0,Croatia,4.0,4.0
7,2004-06-13 19:45:00,France,England,2,1,1.0,1,2004,France,6.0,21.0,England,3.0,19.0
8,2004-06-17 17:00:00,England,Switzerland,3,0,1.0,3,2004,England,3.0,19.0,Switzerland,3.0,3.0
9,2004-06-17 19:45:00,Croatia,France,2,2,0.0,0,2004,Croatia,4.0,4.0,France,6.0,21.0
